In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from sys import getsizeof
import gc
%matplotlib inline

In [2]:
def create_pixels_train_set(expression, path):
    path = path.replace("/","_")
    path = "train_images_cropped/"+str(expression)+"/"+path
    image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    image = image.flatten()
    return list(image)

In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, Activation
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [3]:
base_model = applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(200,200,3), pooling=None)


In [4]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 206, 206, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256                                          
___________________________________________________________________________________________

In [5]:
for layer in base_model.layers[:-75]:
     layer.trainable = False

In [6]:
x = base_model.output
x = Conv2D(8,1)(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)
model = Model(base_model.input, x)

# for layer in base_model.layers[:-75]:
#     layer.trainable = False


In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 206, 206, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256                                          
___________________________________________________________________________________________

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
train_directory = "train_images_cropped_downsampled"
t_size = (200, 200)
b_size = 16
test_directory = "validation_images_cropped_downsampled"

In [10]:
train_gen = ImageDataGenerator(
horizontal_flip = True)

test_gen = ImageDataGenerator(
horizontal_flip = True)

train_generator = train_gen.flow_from_directory(
train_directory,
target_size = t_size,
batch_size = b_size,
class_mode = "categorical")

validation_generator = test_gen.flow_from_directory(
test_directory,
target_size = t_size,
batch_size = b_size,
class_mode = "categorical")



Found 24000 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [11]:
checkpoint = ModelCheckpoint(filepath='saved_models/model.weights.best.down_sampled_2401.hdf5', verbose=1, save_best_only=True)

In [ ]:
history1 = model.fit_generator(train_generator, epochs=150, validation_data=validation_generator, verbose=2, 
                               callbacks=[checkpoint], steps_per_epoch=150, validation_steps=50)

Epoch 1/150
Epoch 00000: val_loss improved from inf to 2.32312, saving model to saved_models/model.weights.best.down_sampled_2401.hdf5
78s - loss: 1.9609 - acc: 0.2704 - val_loss: 2.3231 - val_acc: 0.2700
Epoch 2/150
Epoch 00001: val_loss did not improve
61s - loss: 1.6332 - acc: 0.3862 - val_loss: 2.4803 - val_acc: 0.2888
Epoch 3/150
Epoch 00002: val_loss improved from 2.32312 to 2.20601, saving model to saved_models/model.weights.best.down_sampled_2401.hdf5
62s - loss: 1.5395 - acc: 0.4225 - val_loss: 2.2060 - val_acc: 0.3050
Epoch 4/150
Epoch 00003: val_loss improved from 2.20601 to 1.59846, saving model to saved_models/model.weights.best.down_sampled_2401.hdf5
62s - loss: 1.4904 - acc: 0.4483 - val_loss: 1.5985 - val_acc: 0.3875
Epoch 5/150
Epoch 00004: val_loss improved from 1.59846 to 1.50843, saving model to saved_models/model.weights.best.down_sampled_2401.hdf5
62s - loss: 1.4478 - acc: 0.4517 - val_loss: 1.5084 - val_acc: 0.4313
Epoch 6/150
Epoch 00005: val_loss did not improv